In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import re
import nltk
import string
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
from class_utils.download import download_file_maybe_extract
download_file_maybe_extract("https://github.com/MehmetFiratKomurcu/IMDBReviewClassification/raw/master/imdb_master.csv", directory="data")
nltk.download(['punkt', 'stopwords', 'wordnet'])

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

## Naivný bayesovský klasifikátor

V tomto notebook-u sa budeme venovať naivnému bayesovskému klasifikátoru. Budeme ho aplikovať na jednoduchú úlohou spracovania prirodzeného jazyka. Existuje dátová množina filmových recenzií zoscrapovaná, spoločne s numerickými hodnoteniami, zo [stránky IMDB](https://www.imdb.com/). Numerické hodnotenia boli transformované do dvoch tried, ktoré indikujú pozitívnu alebo negatívnu recenziu. Ukážeme, prečo je naivný bayesovský klasifikátor dobrým kandidátom na riešenie takých úloh aj napriek jeho extrémne zjednodušujúcim predpokladom.

### Načítanie dátovej množiny

Na začiatok načítame dátovú množinu z CSV súboru. Špecifikujeme znakovú sadu ISO-8859-1.



In [ ]:
df = pd.read_csv("data/imdb_master.csv", encoding="ISO-8859-1")
df.head()

Dátová množina obsahuje stĺpec, ktorý vzorky rozdeľuje na tréningové a testovacie. Dáta teda rozdelíme podľa neho týmto preddefinovaným spôsobom.



In [ ]:
df_train = df[df['type'] == 'train']
df_test = df[df['type'] == 'test']

### Predspracovanie textu

Ďalej, keďže naivný bayesovskú klasifikátor nevie pracovať priamo ss textou, budeme každú recenziu potrebovať predspracovať do vektora fixnej dĺžky. Tento proces si prejdeme krok za krokom a pomocou jednej recenzie si budeme ilustrovať, čo sa v ňom presne deje.



In [ ]:
review = df_train['review'].iloc[2]
print(review)

# Odstránime prípadné HTML tagy pomocou regulárnych výrazov.


In [ ]:
html_re = re.compile(r"<[^>]*>")
without_tags = html_re.sub(' ', review)
print(without_tags)

# Odstránime interpunkčné znamienka (náhradou všetkých znakov obsiahnutých v `string.punctuation` prázdnym reťazcom).


In [ ]:
without_punctuation = without_tags

for char in string.punctuation:
    without_punctuation = without_punctuation.replace(char, "")
    
print(without_punctuation)

# Transformujeme všetky písmená na malé.


In [ ]:
lower_case = without_punctuation.lower()
print(lower_case)

# Reťazec rozdelíme na bielych znakoch, aby sme získali jednotlivé slová.


In [ ]:
words = lower_case.split()
print(words)

# Odstránime stop slová (pomocné slová ako "and", "or", "the" a pod.) a čokoľvek, čo sa neskladá výlučne z písmen.


In [ ]:
stop_words = set(stopwords.words('english'))
only_useful_words = [w for w in words if w.isalpha() and not w in stop_words]
print(only_useful_words)

# Slová transformujeme do kánonickej podoby pomocou lemmatizácie.


In [ ]:
lemmatizer = WordNetLemmatizer()
canonical = [lemmatizer.lemmatize(w) for w in only_useful_words]
print(canonical)

# Výsledné slová spojíme späť dokopy.


In [ ]:
preproced_text = " ".join(canonical)
print(preproced_text)

Definujeme funkciu `preproc_text` s tou istou logikou, ktorú následne aplikujeme na každú recenziu v dátovej množine.



In [ ]:
#@title -- function preproc_text -- { display-mode: "form" }
html_re = re.compile(r"<[^>]*>")
lemmatizer = WordNetLemmatizer()

def preproc_text(text):
    text = html_re.sub(' ', text)
    
    # remove punctuation
    for char in string.punctuation:
        text = text.replace(char, "")

    # transform all to lower case
    text = text.lower()

    # split on whitespace
    words = text.split()

    # filter out anything that is not exclusively
    # made of letters or that is in stop words
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if w.isalpha() and not w in stop_words]
    
    # lemmatize the words, turning them into canonical forms
    canonical = [lemmatizer.lemmatize(w) for w in words]
    
    # join the words back together
    preproced_text = " ".join(canonical)
    return preproced_text

In [ ]:
reviews_train = [preproc_text(text) for text in df_train['review']]
reviews_test = [preproc_text(text) for text in df_test['review']]

### Transformácia textu na vektor fixnej dĺžky

#### Bag of words (batoh slov)

Hoci sme už na texte vykonali veľa predspracovania, stále ho máme vo forme reťazca a nie vektora fixnej veľkosti. Ako ho teda vektorizujeme? Jeden spôsob je vytvoriť reprezentáciu **bag of words**  (batoh slov): jednoducho spočítať kooľko krát sa každé slovo v recenzii nachádza. Toto realizuje trieda `CountVectorizer` z balíčka scikit learn.



In [ ]:
count_vectorizer = CountVectorizer()
bag_of_words = count_vectorizer.fit_transform(reviews_train)
bag_of_words.shape

Bag of words je obrovská matica: s jedným stĺpcom pre každé unikátne slovo. Práve preto sme venovali toľko úsilia transformácii slov na ich kánonické formy: inak by matice boli ešte niekoľkonásobne väčšie. Platí tiež, že každá recenzia obsahuje len zlomok všetkých možných slov a väčšina prvkov teda bude nulová. Z tohto dôvodu sa bag of words reprezentácie ukladajú vo forme riedkych (sparse) matíc: uložia sa len hodnoty nenulových prvkov.

#### Bag of N-grams (batoh n-gramov)

Vo všeobecnosti platí, že okrem prítomnosti jednotlivých slov nás zaujíma aj ich vzájomné poradie alebo ich špecifické kombinácie. Aby sme vedeli tieto aspekty zachytiť, môžeme použiť takzvané **n-gramy** : budeme sa na slová pozerať v rámci ich n-slovných komntextov a počítať ich výskyty namiesto výskytov slov. Takto by sme to realizovali pre 2-gramy:



In [ ]:
count_vectorizer = CountVectorizer(ngram_range=(2, 2))
grams_2 = count_vectorizer.fit_transform(reviews_train)
grams_2.shape

Všimnite si, že matica má teraz omnoho viac stĺpcov než predtým. Je to samozrejme preto, že 2-gramových kombinácií je omnoho viac než samotných jednotlivých slov. Našťastie nie všetky kombinácie slov sa v textoch vyskytujú, takže počet 2-gramov nebude druhou mocninou počtu slov.

Ak by sme chceli sledovať výskyty jednotlivých slov aj 2-gramov, môžeme tiež špecifikovať `ngram_range=(1, 2)` – takto aj naozaj budeme robiť.

#### TF-IDF

Napokon treba brať do úvahy to, že existujú bežné slová (n-gramy), ktoré sa vyskytujú vo veľmi veľkom počte dokumentov. Intuícia hovorí, že tieto budú pri rozlišovaní medzi triedami asi menej užitočné a preto nechceme, aby mali pri predikcii disproporčne vysoký vplyv. Preto namiesto jednoduchých počtov výskytov vypočítame **TF-IDF** : t.j. **term-frequency times inverse document-frequency**  (frekvenciu pojmu krát inverznú dokumentovú frekvenciu). Nahrubo povedané, frekvenciu (počet výskytov) každého pojmu budeme deliť celkovým počtom dokumentov, v ktorých sa vyskytuje.

Presnejšie povedané, ak frekvenciu (počet výskytov) pojmu $t$ v dokumente $d$ označíme $\text{tf}(t, d)$, inverzná dokumentová frekvencia je definovaná takto [TfidfTransformer](#TfidfTransformer):

$$
\text{idf}(t) = \log \left[ \frac{1 + n}{1 + \text{df}(t)} \right] + 1,
$$
kde $n$ je celkový počet dokumentov a $\text{df}(t)$ je počet dokumentov obsahujúcich pojem $t$. TF-IDF je potom jednoducho [TfidfTransformer](#TfidfTransformer):

$$
\text{tf-idf}(t, d) = \text{tf}(t, d) \cdot \text{idf}(t).
$$
Na to, aby sme TF-IDF získali V Python-e, použijeme z balíčka scikit learn jednoducho namiesto triedy `CountVectorizer` triedu `TfidfVectorizer`. Použime teda teraz `TfidfVectorizer` na transformáciu našich recenzií na `X_train` a `X_test`.



In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2))

X_train = vectorizer.fit_transform(reviews_train)
Y_train = df_train['label']

X_test = vectorizer.transform(reviews_test)
Y_test = df_test['label']

### Tréning modelu

Teraz, keď sme predspracovali dáta, môžeme na nich natrénovať model. Ako sme videli, TF-IDF vektory sú dosť veľké: pri ich ukladaní dokonca preferujeme použitie riedkych matíc. Dôvod prečo naivný bayesovský klasifikátor nie je zlou voľbou pre takéto úlohy (napriek dosť extrémnym zjednodušujúcim predpokladom), je, že by bolo o dosť ťažšie natrénovať na dátach takých rozmerov komplexnejší model. V minulosti, s menej výkonným hardvérom, to často nebolo realistické, a vyhodnejšie to môže byť v niektorých prípadoch aj dnes – za predpokladu, že sú výsledky dostatočne dobré.

Platí tiež, že každá metóda, ktorej tréning na našej dátovej množine má byť rýchly, by mala mať podporu pre riedke matice (napr. rozhodovacie stromy v balíčku scikit learn ju nemajú): ak sa ich bude snažiť konvertovať do hustej reprezentáciu, tréning bude trvať podstatne dlhšie. V balíčku scikit learn však existuje ešte zopár iných jednoduchých metód, ktoré majú podporu pre riedke matice – napríklad logistická regresia. Tie by nemalo byť omnoho ťažšie natrénovať na tých istých dátach.

V každom prípade, teraz si pomocou triedy `MultinomialNB` vytvoríme a natrénujeme naivný bayesovský klasifikátor:



In [ ]:
model = MultinomialNB()
model.fit(X_train, Y_train)

### Testovanie

Napokon sa pozrime, akú má náš model správnosť.



In [ ]:
y_test = model.predict(X_test)

cm = pd.crosstab(Y_test, y_test,
                 rownames=['actual'],
                 colnames=['predicted'])
print(cm, "\n")

acc = accuracy_score(Y_test, y_test)
print("Accuracy = {}".format(acc))

### References

<a id="TfidfTransformer">[TfidfTransformer]</a> sklearn.feature_extraction.text.TfidfTransformer. [https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer).

